In [1]:
import transformers

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

/home/mirac13/anaconda3/envs/tf2/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201029). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print("Invalid device or cannot modify virtual devices once initialized.")
    pass

In [3]:
tf.__version__

'2.5.0-dev20201029'

In [4]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [5]:
val_df = pd.read_json('data/dev_unseen.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]

In [6]:
img_width = 224
img_height = 224

In [7]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [8]:
max_len = 128

### Train

In [9]:
def build_model():
    
    encoder = transformers.TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    encoder.trainable = False

    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    input_ids = tf.keras.layers.Input(name = "input_ids", shape = (max_len,), dtype = tf.int32)

    attention_mask = tf.keras.layers.Input(name = "attention_mask", shape = (max_len,), dtype = tf.int32)

    y = encoder(input_ids, attention_mask = attention_mask)[0]
    y = tf.keras.layers.BatchNormalization()(y)
    
    y = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True))(y)
    y = tf.keras.layers.BatchNormalization()(y)

    extractor = tf.keras.applications.EfficientNetB0(include_top = False, input_tensor = input_img, weights = None)
    
    extractor.trainable = True
    
    x = tf.keras.layers.BatchNormalization()(extractor.output)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    y = tf.keras.layers.GlobalMaxPooling1D()(y)
    y = tf.keras.layers.BatchNormalization()(y)
    
    x = tf.keras.layers.concatenate([x, y])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Flatten()(x)
    
    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation = "relu"))(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tfa.layers.WeightNormalization(tf.keras.layers.Dense(256, activation = "relu"))(x)
    
    x = tf.keras.layers.Dropout(0.25)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    out = tfa.layers.WeightNormalization(tf.keras.layers.Dense(1, activation = 'sigmoid'))(x)

    model = tf.keras.models.Model([input_img, input_ids, attention_mask], out)

    return model

In [10]:
model = build_model()
model.compile(optimizer = tfa.optimizers.LazyAdam(0.001), loss = "binary_crossentropy", metrics = ["accuracy", tf.keras.metrics.AUC(name = 'auc')])
model.summary()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [11]:
def encode_single_sample(img_path, label, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf', max_length = 128, padding = 'max_length')
    return {"image": img, "label": label, "text": text}

In [12]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        z = np.zeros((self.batch_size), dtype = "float32")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_text):
            sample = encode_single_sample(j,k,l)
            w[i] = sample["image"].numpy().tolist()
            x[i] = sample["text"]['input_ids'].numpy().tolist()[0]
            y[i] = sample["text"]['attention_mask'].numpy().tolist()[0]
            z[i] = sample["label"]
        
        return [w,x,y], z

In [13]:
train_gen = HatefulMemes(8, train_df["img"].values.tolist(), train_df["label"].values.tolist(), \
                         train_df["text"].values.tolist())
val_gen = HatefulMemes(8, val_df["img"].values.tolist(), val_df["label"].values.tolist(), \
                         val_df["text"].values.tolist())

In [14]:
epochs = 50
early_stopping_patience = 10

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_auc", patience = early_stopping_patience, restore_best_weights = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.2,
                                                 patience = 5, min_lr = 0.0001)

history = model.fit(train_gen, validation_data = val_gen, epochs = epochs, callbacks = [early_stopping,                                        reduce_lr])

Epoch 1/50
1062/1062 [==============================] - 873s 812ms/step - loss: 0.6525 - accuracy: 0.6246 - auc: 0.6120 - val_loss: 0.9687 - val_accuracy: 0.6418 - val_auc: 0.5747
Epoch 2/50
1062/1062 [==============================] - 847s 798ms/step - loss: 0.5789 - accuracy: 0.7050 - auc: 0.7217 - val_loss: 0.6898 - val_accuracy: 0.6269 - val_auc: 0.6172
Epoch 3/50
1062/1062 [==============================] - 841s 792ms/step - loss: 0.5336 - accuracy: 0.7435 - auc: 0.7769 - val_loss: 0.7277 - val_accuracy: 0.6381 - val_auc: 0.5948
Epoch 4/50
1062/1062 [==============================] - 846s 797ms/step - loss: 0.5278 - accuracy: 0.7455 - auc: 0.7873 - val_loss: 0.7377 - val_accuracy: 0.6231 - val_auc: 0.5867
Epoch 5/50
1062/1062 [==============================] - 843s 793ms/step - loss: 0.5005 - accuracy: 0.7670 - auc: 0.8104 - val_loss: 0.8295 - val_accuracy: 0.6343 - val_auc: 0.6057
Epoch 6/50
1062/1062 [==============================] - 844s 794ms/step - loss: 0.4737 - accuracy: 0

In [15]:
for layer in model.layers:
    if(layer.name.startswith('tf_distil_bert_model')):
        layer.trainable = True
model.compile(optimizer = tfa.optimizers.LazyAdam(0.001), loss = "binary_crossentropy", metrics = ["accuracy", tf.keras.metrics.AUC(name = 'auc')])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           image[0][0]                      
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

Total params: 75,031,977
Trainable params: 73,140,094
Non-trainable params: 1,891,883
__________________________________________________________________________________________________


In [16]:
epochs = 50
early_stopping_patience = 5

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_auc", patience = early_stopping_patience, restore_best_weights = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.2,
                                                 patience = 2, min_lr = 0.0001)

history_new = model.fit(train_gen, validation_data = val_gen, epochs = epochs, callbacks = [early_stopping,                                        reduce_lr])

Epoch 1/50
1062/1062 [==============================] - 947s 882ms/step - loss: 0.6719 - accuracy: 0.6216 - auc: 0.5040 - val_loss: 0.6649 - val_accuracy: 0.6269 - val_auc: 0.4888
Epoch 2/50
1062/1062 [==============================] - 929s 875ms/step - loss: 0.6540 - accuracy: 0.6462 - auc: 0.4966 - val_loss: 0.6799 - val_accuracy: 0.6194 - val_auc: 0.4859
Epoch 3/50
1062/1062 [==============================] - 930s 876ms/step - loss: 0.6485 - accuracy: 0.6499 - auc: 0.5188 - val_loss: 0.6694 - val_accuracy: 0.6269 - val_auc: 0.4909
Epoch 4/50
1062/1062 [==============================] - 931s 877ms/step - loss: 0.6573 - accuracy: 0.6366 - auc: 0.5064 - val_loss: 0.7004 - val_accuracy: 0.6269 - val_auc: 0.4905
Epoch 5/50
1062/1062 [==============================] - 928s 873ms/step - loss: 0.6494 - accuracy: 0.6480 - auc: 0.5149 - val_loss: 0.6807 - val_accuracy: 0.6269 - val_auc: 0.4533
Epoch 6/50
1062/1062 [==============================] - 926s 872ms/step - loss: 0.6523 - accuracy: 0

### Predict

In [ ]:
train_df = pd.read_json('data/test_unseen.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]

In [ ]:
def val_encode_single_sample(img_path, text):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.int32)
    img = tf.image.resize(img, [img_height, img_width])
    text = tokenizer(text, return_tensors = 'tf', max_length = 128, padding = 'max_length')
    return {"image": img, "text": text}

In [ ]:
class ValHatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        w = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        x = np.zeros((self.batch_size,max_len,), dtype = "int32")
        y = np.zeros((self.batch_size,max_len,), dtype = "int32")
        
        for i,j,k in zip(range(self.batch_size), batch_input_img_paths, batch_text):
            sample = val_encode_single_sample(j,k)
            w[i] = sample["image"].numpy().tolist()
            x[i] = sample["text"]['input_ids'].numpy().tolist()[0]
            y[i] = sample["text"]['attention_mask'].numpy().tolist()[0]
        
        return [w,x,y]

In [ ]:
pred_gen = ValHatefulMemes(1, train_df["img"].values.tolist(), train_df["text"].values.tolist())

In [ ]:
preds = model.predict(pred_gen, verbose = 1)

In [ ]:
def prob2pred(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [ ]:
train_df["proba"] = [i[0] for i in preds.tolist()]
train_df["label"] = train_df["proba"].apply(prob2pred)

In [ ]:
train_df = train_df.drop(["img","text"], axis = 1)
train_df

In [ ]:
train_df.to_csv("submission_new.csv", index = False)